In [10]:
import os 


In [11]:
%pwd 


'/Users/udita19/Desktop/Projects/BrainTumorClassification/research'

In [12]:
cd ..

/Users/udita19/Desktop/Projects/BrainTumorClassification


/Users/udita19/Desktop/Projects/BrainTumorClassification/Tumorenv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
%pwd

'/Users/udita19/Desktop/Projects/BrainTumorClassification'

In [14]:
import tensorflow as tf 

In [15]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

[2024-03-11 00:00:50,666: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]


In [16]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path 
    training_data: Path 
    all_params: dict 
    params_image_size: list 
    params_batch_size: int 

In [17]:
from TumorClassifier.constants import * 
from TumorClassifier.utils.common import read_yaml, create_directories, save_json

In [22]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/brain_tumor",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [23]:
from urllib.parse import urlparse

In [24]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

In [25]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-03-11 00:01:21,052: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-11 00:01:21,054: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-11 00:01:21,054: INFO: common: created directory at: artifacts]
[2024-03-11 00:01:21,491: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
Found 777 images belonging to 2 classes.
49/49 [==============================] - 237s 5s/step - loss: 0.2653 - accuracy: 0.9498
[2024-03-11 00:05:18,682: INFO: common: json file saved at: scores.json]
